# Downline Braking Investigation

In [1]:
from flightdata import Flight, Fields, CIDTypes
from geometry import Point, Coord
from math import pi
import numpy as np
import pandas as pd 
import plotly.graph_objects as go

def plot2d(datax, datay, colour='black', width=1, fig=None, name=None):
    if not fig:
        fig=go.Figure()
    fig.add_trace(go.Scatter(x=datax, y=datay, name=name, line=dict(color=colour, width=width)))
    fig.update_layout(width = 800, height = 500, margin=dict(l=20, r=20, t=20, b=20),)
    fig.update_yaxes(scaleanchor = "x",scaleratio = 1)
    return fig

def plot3d(datax, datay, dataz, colour='black',fig=None, width=1):
    if not fig:
        fig=go.Figure()
    fig.add_trace(go.Scatter3d(x=datax, y=datay, z=dataz, line=dict(color=colour, width=width),
        mode = 'lines'))
    fig.update_layout(width = 800, height = 500, margin=dict(l=20, r=20, t=20, b=20))
    fig.update_yaxes(scaleanchor = "x",scaleratio = 1)
    return fig

def rotate_data(bin, angle):
    # get a vector pointing in the desired x axis by rotating the default x axis vector by the angle requested.
    box_normal_vector = Point(1, 0, 0).rotate(Point(0,0,angle).to_rotation_matrix())

    #create a new Coord based on the origin (unchanged), the z axis (up), and the new x axis vector. 
    box_cid = Coord.from_zx(Point(0, 0, 0), Point(0, 0, -1), box_normal_vector)

    def point_to_box(x, y, z):
        box_point = Point(x, y, z).rotate(box_cid.rotation_matrix)
        return box_point.x, box_point.y, box_point.z

    def euler_to_box(x, y, z):
        seupoint = Point(x, y - pi, - z - angle)
        return seupoint.x, seupoint.y, seupoint.z

    transforms = {i: lambda *args: args  for i in range(0, 7)}
    transforms[CIDTypes.CARTESIAN] = point_to_box
    transforms[CIDTypes.EULER] = euler_to_box
    transforms[CIDTypes.ZONLY] = lambda *args: tuple(-arg for arg in args) 
    transforms[CIDTypes.XY]: lambda *args: point_to_box(*args, z=0) 
    
    return bin.transform(transforms)



bin=Flight.from_log('./braking/00000134.BIN')
flight = rotate_data(bin, -126*pi/180)

 0x11 at 10864418
bad header 0x11 0x90 at 10864419
bad header 0x90 0x89 at 10864420
bad header 0x89 0x10 at 10864421
bad header 0x10 0x00 at 10864422
bad header 0x00 0x00 at 10864423
bad header 0x00 0x00 at 10864424
bad header 0x00 0x00 at 10864425
bad header 0x00 0x18 at 10864426
bad header 0x18 0x00 at 10864427
bad header 0x00 0x05 at 10864428
bad header 0x05 0x00 at 10864429
bad header 0x00 0xe6 at 10864430
bad header 0xe6 0xff at 10864431
bad header 0xff 0xf7 at 10864432
bad header 0xf7 0xff at 10864433
bad header 0xff 0x00 at 10864434
bad header 0x00 0x00 at 10864435
bad header 0x00 0x04 at 10864436
bad header 0x04 0x00 at 10864437
bad header 0x00 0x00 at 10864438
bad header 0x00 0x00 at 10864439
bad header 0x00 0x00 at 10864440
bad header 0x00 0x00 at 10864441
bad header 0x00 0x00 at 10864442
bad header 0x00 0x00 at 10864443
bad header 0x00 0x37 at 10864444
bad header 0x37 0xfe at 10864445
bad header 0xfe 0x00 at 10864446
bad header 0x00 0x00 at 10864447
bad header 0x00 0xa3 at 1

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x98 in position 1: invalid start byte

A flight was performed consisting of many vertical lines with different throttle settings. The figure below shows a position trace of the flight.

In [ ]:
plot3d(*flight.read_field_tuples(Fields.POSITION)).show()

Below the relevant data for analysis is extracted from the log:

In [ ]:
from math import sqrt

M = 4.4

rpm_data = flight.read_fields([Fields.RPM, Fields.TXCONTROLS, Fields.POSITION, Fields.VELOCITY])
rpm_data = rpm_data[['rpm_0', 'tx_controls_0'] + Fields.POSITION.names + Fields.VELOCITY.names]
rpm_data['rpm_0'] = rpm_data['rpm_0'] / 280
rpm_data.columns = 'rpm,throttle,x,y,z,vx,vy,vz'.split(',')

rpm_data['v'] = np.vectorize(lambda x,y,z: sqrt(x**2 + y**2 + z**2))(rpm_data.vx, rpm_data.vy, rpm_data.vz)
diffs = rpm_data.v.diff()
steps = pd.Series(rpm_data.index).diff()
v = np.vectorize(lambda n, d: n / d)(diffs, steps)
rpm_data['a'] = v

rpm_data['body_x_force'] = M * rpm_data.a

rpm_data.sample(10)



Initial tests were performed on the ground, to see RPM response to step changes input:

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

static_rpmdata = rpm_data.loc[111: 120]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=static_rpmdata.index, y=static_rpmdata.throttle, name='throttle'),secondary_y=False)
fig.add_trace(go.Scatter(x=static_rpmdata.index, y=static_rpmdata.rpm, name='rpm'),secondary_y=True)
fig.update_xaxes(title_text="time")
fig.update_yaxes(title_text="throttle", secondary_y=False)
fig.update_yaxes(title_text="rpm", secondary_y=True)
fig.show()


fig=go.Figure()
fig.add_trace(go.Scatter(x=static_rpmdata.throttle, y=static_rpmdata.rpm, mode='markers'))
fig.update_layout(width = 800, height = 500, margin=dict(l=20, r=20, t=20, b=20),)
fig.update_xaxes(title_text="throttle")
fig.update_yaxes(title_text="rpm")

fig.show()

Next split the flight into individual up and downlines:

In [ ]:
lines = [
    [196, 208], [213,216], [223,228], [234,239], [244,252], [258,263], [268,278], [284,290], [294,309], [314,320], [326,336], [342,347],
    [352,364],[370,376],[381,391],[396,403],[408,419],[423,429],[434,444],[450,455],[460,470],[475,480],[485,495],[500,505],[510,519],[524,529],[534,542],
    [548,552],[558,566],[572,577]
]

class Line():
    def __init__(self, data):
        self.data = data
        self.throttle = data.throttle.mean()
        self.rpm = data.rpm.mean()
        self.func = None
        if self.rpm < 60:
            self.rpm = 0

    def isupline(self):
        return self.data.iloc[-1].z > self.data.iloc[0].z

    def get_prop_drag(self, u):
        return self.func[0] * u**3 + self.func[1] * u**2 + self.func[2] * u + self.func[3]


uplines = []
upline_refs = []
downlines = []
downline_refs = []
for line in lines:
    lin = Line(rpm_data.loc[line[0]:line[1]])
    if lin.isupline():
        uplines.append(lin)
        upline_refs.append([len(uplines) -1, lin.throttle, lin.rpm])
    else:
        downlines.append(lin)
        downline_refs.append([len(uplines)-1, lin.throttle, lin.rpm])

downline_refs = pd.DataFrame(downline_refs, columns='id,throttle,rpm'.split(',')).sort_values('rpm')
upline_refs = pd.DataFrame(upline_refs, columns='id,throttle,rpm'.split(','))

downline_refs

Two tests were performed with the prop stopped, these can be used to calculate Cd0:

In [ ]:

rho = 1.225
S = 0.15*2 # Guessed wing area, need to measure it

def get_downline_cd0(u, f):
    return (M * 9.81 - f) / (0.5 * rho * u**2 * S)

prop_stopped_tests = downline_refs.loc[downline_refs.rpm == 0]


fig=go.Figure()

temp = []
for id in prop_stopped_tests.id.to_list():
    test = downlines[id]
    test.data['cd0'] = np.vectorize(get_downline_cd0)(test.data.v, test.data.body_x_force) 
    
    temp.append(test.data['cd0'].mean())
    print('cd0 from test ', id, ' = ', test.data['cd0'].mean())
    print('standard deviation in cd0 from test ', id, ' = ', test.data['cd0'].std())

    fig.add_trace(go.Scatter(x=test.data.v, y=test.data.cd0))

Cd0 = sum(temp) / len(prop_stopped_tests.id.to_list())

fig.update_layout(width = 800, height = 500, margin=dict(l=20, r=20, t=20, b=20),)
fig.update_xaxes(title_text="u (m/s)")
fig.update_yaxes(title_text="Cd0")

fig.show()

Now that Cd0 has been estimated the additional contribution to drag from the prop turning can be calculated for the other tests

In [ ]:
def get_drag_force(u):
    return 0.5 * rho * u**2 * S * Cd0

def get_downline_prop_drag(u, f):
    return 9.81 * M - f - get_drag_force(u)


fig = go.Figure()
meandata = []
for id in downline_refs.id.to_list():
    test = downlines[id]
    test.data['prop_drag'] = np.vectorize(get_downline_prop_drag)(test.data.v, test.data.body_x_force)

    test.func = np.polyfit(test.data.v, test.data.prop_drag,3)

    test.data['simple_prop_drag'] = np.vectorize(test.get_prop_drag)(test.data.v)

    fig.add_trace(go.Scatter(x=test.data.v, y=test.data.simple_prop_drag, name=int(test.rpm)))
fig.update_layout(title= 'Propeller Braking Force vs Airpseed for a Range of RPMs')
fig.update_xaxes(title_text="airspeed (m/s)")
fig.update_yaxes(title_text="prop braking force (N)")
fig.update_layout(yaxis_range=[-5, 10], xaxis_range=[33, 40])
fig.update_layout(width = 800, height = 600, margin=dict(l=20, r=20, t=40, b=20),)
fig.show()

In [ ]:
print('Cd0 = ', str("{:.2f}".format(no_power.Cd0.mean())))